This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [20]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case

#API_KEY = 'your-key-here'

# Import QUANDL_API_KEY from the .env file
from dotenv import load_dotenv
load_dotenv(verbose=True)

import os
API_KEY = os.getenv("QUANDL_API_KEY")

print(type(API_KEY))
#print(API_KEY)


<class 'str'>


Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [21]:
# First, import the relevant modules
import requests
from datetime import datetime, timedelta


In [22]:
# Make a sample API Call for 1 day worth of data

#get yesterday's date with datetime and timedelta modules
yesterday = datetime.now() - timedelta(days=1)

In [23]:
# set API call parameters
database_code = 'FSE' # Frankfurt Stock Exchange Database
dataset_code = 'AFX_X' #Ticker symbol for Carl Zeiss Meditec
start_date = yesterday.strftime('%Y-%m-%d') #convert to string and format to yyyy-mm-dd

url = 'https://www.quandl.com/api/v3/datasets/'+database_code+'/'+dataset_code+'/data.json'

payload = {'api_key': API_KEY, 'start_date': start_date}

r = requests.get(url, params=payload)

In [24]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r_dict = r.json()
#print(r_dict)
#print(r_dict['dataset_data'])
print(r_dict['dataset_data']['column_names'])
#print(r_dict['dataset_data']['data'])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).



In [25]:
#Collect Data for 2017
# set API call parameters
database_code = 'FSE' # Frankfurt Stock Exchange Database
dataset_code = 'AFX_X' #Ticker symbol for Carl Zeiss Meditec
start_date = '2017-01-01'
end_date = '2017-12-31'

url = 'https://www.quandl.com/api/v3/datasets/'+database_code+'/'+dataset_code+'/data.json'

payload = {'api_key': API_KEY, 'start_date': start_date, 'end_date': end_date}

r = requests.get(url, params=payload)

2. Convert the returned JSON object into a Python dictionary.

In [26]:
# save the respnse data to a dict
d = r.json()

In [27]:
lst = d['dataset_data']['data'] #extract the List of lists Data from the dict for processing
#print(lst)

In [28]:
#extract relevant variables opening, high, low, closing, trading volume respectively at colums of indices 1,2,3,4,6 of the list
opening = []
closing = []
high = []
low = []
trading_vol = []

for l in lst :
    #print(l[1])
    opening.append(l[1])  
    high.append(l[2])
    low.append(l[3])
    closing.append(l[4])
    trading_vol.append(l[6])


3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [29]:
# We observe that opening has some missing values while closing doesn't
#print(opening)
#print(closing)


In [30]:
# This operation is redundant since I realized I was looking at the wrong timeframe which had None values
# I'll keep it so if I need to change the timeframe I can still use this code.
# How do we handle missing values?
# We will replace the missing values in opening with the closing value for the same day.


clean_opening = []
for idx, val in enumerate(opening):
    if val is None:
        clean_opening.append(closing[idx])
    else :
        clean_opening.append(val)


In [31]:
# Verify that this worked correctly
#print(clean_opening)
#print(closing)


In [32]:
highest_opening = max(clean_opening)
lowest_opening = min(clean_opening)
print('The higest opening price for ', dataset_code, ' in 2017 is: ', highest_opening)
print('The lowest opening price for ', dataset_code, ' in 2017 is: ', lowest_opening)

The higest opening price for  AFX_X  in 2017 is:  53.11
The lowest opening price for  AFX_X  in 2017 is:  34.0


4. What was the largest change in any one day (based on High and Low price)?

In [33]:
hi_lo_change = []
for idx, val in enumerate(high):
    hi_lo_change.append(val-low[idx])
    
max_hi_lo_change = max(hi_lo_change)
print('The largest daily change in any one day (based on High and Low price) for ', dataset_code, ' in 2017 is: ', round(max_hi_lo_change,2))

The largest daily change in any one day (based on High and Low price) for  AFX_X  in 2017 is:  2.81


5. What was the largest change between any two days (based on Closing Price)?

In [34]:
day_daybefore_change = []
for idx, val in enumerate(closing):
    if idx == 0 :
        day_daybefore_change.append(0)
    else :
        day_daybefore_change.append(val-closing[idx-1])
        
max_day_daybefore_change = max(day_daybefore_change)
print('The largest change between any two days (based on Closing Price) for ', dataset_code, ' in 2017 is: ', round(max_day_daybefore_change,2))

The largest change between any two days (based on Closing Price) for  AFX_X  in 2017 is:  2.56


6. What was the average daily trading volume during this year?

In [35]:
# This operation is redundant since I realized I was looking at the wrong timeframe which had None values
# I'll keep it so if I need to change the timeframe I can still use this code.
#Trading vol has some None values too. We will replace them with zero. 
#print(trading_vol)
clean_trading_vol = []
for idx, val in enumerate(trading_vol):
    if val is None:
        clean_trading_vol.append(0)
    else :
        clean_trading_vol.append(val)

In [36]:
avg_daily_trade_vol = sum(clean_trading_vol) / len(clean_trading_vol)
print('The average daily trading volume for ', dataset_code, ' in 2017 is: ', round(avg_daily_trade_vol,2))

The average daily trading volume for  AFX_X  in 2017 is:  89124.34


7. What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [37]:
import statistics
median_daily_trade_vol = statistics.median(clean_trading_vol)
print('The median trading volume for ', dataset_code, ' in 2017 is: ', round(median_daily_trade_vol,2))

The median trading volume for  AFX_X  in 2017 is:  76286.0
